In [1]:
import nltk, re, pprint
import pandas as pd
import numpy as np

In [2]:
#data = pd.read_csv(r"D:\ML\Datasets\tcc_ceds_music.csv")
data = pd.read_csv("D:\ML\Datasets\labeled_lyrics_cleaned.csv")

In [3]:
data.head()

,Unnamed: 0,artist,seq,song,label
0,0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday,0.626
1,1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die,0.630
2,2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside,0.240
3,3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot,0.536
4,4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds,0.371


In [4]:
data.drop(labels="Unnamed: 0", axis=1, inplace=True)
data.rename(columns={"seq": "lyrics", "label": "valency"}, inplace=True)

#data = data.drop_duplicates(subset=['lyrics', 'song'])
data.sort_values(by=['song', 'valency'], ascending=False, inplace=True) # to keep highest value valency 
data = data.drop_duplicates(subset='lyrics')
pd.set_option('display.float_format', lambda x: '%.2f' % x) # round everything to 2 decimal places
data = data.reset_index() 
data.head()

,index,artist,lyrics,song,valency
0,54911,Simon & Milo,"Hello, this is Stacy, the computer\nGood morni...",www.nevergetoveryou,0.68
1,82479,Hippo Campus,See how the western kids\r\nHave silicon insid...,western kids,0.52
2,82478,Hippo Campus,"Wisconsin pines, collaborating with the day gl...",way it goes,0.52
3,82477,Hippo Campus,"I see meaning where you don't, where you don't...",vines,0.66
4,82476,Hippo Campus,My thoughts are a battlefield of sub-surreal a...,vacation,0.55


In [35]:
from nltk import tokenize
from nltk.corpus import brown, stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import webtext

lyrics = [x.replace("\n", " ").strip().replace("\r", "") for x in data['lyrics']]

In [6]:
#lyrics

In [174]:
import string # removing punctuation

song = data['lyrics'][480].replace("\n", " ").replace("\r", "")
#.replace("  ", " ").strip()
#.replace("  ", " ").strip()
#song = data['lyrics'][0]
#song = song.translate(str.maketrans('', '', string.punctuation))
song

"It's the humming of the engines On the Greyhound bus and trains That keeps your memory on my mind And here with me Many miles have come between us Cold nights on some roadside Bringing back Your gentle ways of loving me  It's the thought of how you love me How you treat me, how you thrill me That clears my fogged-up mind To where I see Some little girl, as I walk by Just looking at me sadly Bringing back Your gentle ways of loving me  It's just your gentle ways of loving me It's just your gentle ways of loving me  It's the happiness I've never found With other loves that bothers And a mother saying How her child loves me And times you never talked about Those late evening hours Bringing back Your gentle ways of loving me  It's just your gentle ways of loving me It's just your gentle ways of loving me It's just your gentle ways of loving me It's just your gentle ways of loving me "

In [163]:
from nnsplit import NNSplit

splitter = NNSplit.load("en")

In [175]:
splits = splitter.split([song.lower()])[0]
song_sents = [str(x).strip() for x in splits]
song_sents

["it's the humming of the engines on the greyhound bus and trains that keeps your memory on my mind and here with me",
 'many miles have come between us cold nights on some roadside bringing back your gentle ways of loving me',
 "it's the thought of how you love me how you treat me, how you thrill me that clears my fogged-up mind to where i see some little girl, as i walk by just looking at me sadly bringing back your gentle ways of loving me",
 "it's just your gentle ways of loving me",
 "it's just your gentle ways of loving me",
 "it's the happiness i've never found with other loves that bothers and a mother saying how her child loves me and times you never talked about those late evening hours bringing back your gentle ways of loving me",
 "it's just your gentle ways of loving me",
 "it's just your gentle ways of loving me",
 "it's just your gentle ways of loving me",
 "it's just your gentle ways of loving me"]

In [178]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()    
tokens = [tokenize.word_tokenize(sent) for sent in song_sents]
tokens

[['it',
  "'s",
  'the',
  'humming',
  'of',
  'the',
  'engines',
  'on',
  'the',
  'greyhound',
  'bus',
  'and',
  'trains',
  'that',
  'keeps',
  'your',
  'memory',
  'on',
  'my',
  'mind',
  'and',
  'here',
  'with',
  'me'],
 ['many',
  'miles',
  'have',
  'come',
  'between',
  'us',
  'cold',
  'nights',
  'on',
  'some',
  'roadside',
  'bringing',
  'back',
  'your',
  'gentle',
  'ways',
  'of',
  'loving',
  'me'],
 ['it',
  "'s",
  'the',
  'thought',
  'of',
  'how',
  'you',
  'love',
  'me',
  'how',
  'you',
  'treat',
  'me',
  ',',
  'how',
  'you',
  'thrill',
  'me',
  'that',
  'clears',
  'my',
  'fogged-up',
  'mind',
  'to',
  'where',
  'i',
  'see',
  'some',
  'little',
  'girl',
  ',',
  'as',
  'i',
  'walk',
  'by',
  'just',
  'looking',
  'at',
  'me',
  'sadly',
  'bringing',
  'back',
  'your',
  'gentle',
  'ways',
  'of',
  'loving',
  'me'],
 ['it', "'s", 'just', 'your', 'gentle', 'ways', 'of', 'loving', 'me'],
 ['it', "'s", 'just', 'your', 